In [24]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
from cloudant.view import View
import sys



#db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
#db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
#db3 = CloudantDB("tweets_raw")         # raw tweet data in the specified locations
#print(len(db1.get_data()))
#print(db2.get_data(n=10000))
#print(len(db3.get_data(n=10)))


def sentiment_zone():
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    # get sentiment stats by zone
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc, 'view_zone')
    summary = []
    for row in view(group=True)['rows']:
        summary.append(row)
    
    db1.close()
    db2.close()
    return summary

    
def dataofday(date):    
    # get all data from a specific day
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    n = 100  # number of tweets need to retrieve
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    data = []
    for row in view(group=False,limit=n,reduce=False,key=date,include_docs=True)["rows"]:
        data.append(row)
    
    db1.close()
    db2.close()
    return data

    
    
def sentiment_day(date):
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    # get sentiment stats by day
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    sentiment_day = []
    for row in view(group=True,reduce=True)["rows"]:
        sentiment_day.append(row)
    db1.close()
    db2.close()
    return sentiment_day

    


if __name__ == "__main__":
    ipaddress=sys.argv[1]
    url="http://"+ipaddress
    sentimental_zone=sentiment_zone(ipaddress)
    print(sentimental_zone)
    date = "2021-01-01"
    data=dataofday(date)

    sentiofday=sentiment_day(date,ipaddress)
    print(sentiofday)

TypeError: sentiment_zone() takes 0 positional arguments but 1 was given

In [18]:
!pip install cloudant

In [19]:
from database_cloudant import CloudantDB

In [21]:
db1 = CloudantDB("tweets_keywords") 

db2 = CloudantDB("tweets_no_keywords") 

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_keywords


In [22]:
print(sentiment_zone())

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_keywords
Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords
[{'key': 'Adelaide', 'value': {'sum': 3252.0666369274995, 'count': 4687, 'min': 0.0018489063, 'max': 0.9991077, 'sumsqr': 2633.231996606998}}, {'key': 'Ballarat', 'value': {'sum': 215.24225406899998, 'count': 301, 'min': 0.007695496, 'max': 0.996231, 'sumsqr': 179.6833548591572}}, {'key': 'Brisbane', 'value': {'sum': 6112.858656253502, 'count': 8605, 'min': 0.0015777647, 'max': 0.9991449, 'sumsqr': 5026.036582776077}}, {'key': 'Bunbury', 'value': {'sum': 16.805452449999997, 'count': 21, 'min': 0.12071696, 'max': 0.9948834, 'sumsqr': 14.993714870919584}}, {'key': 'Canberra', 'value': {'sum': 1852.1211890105, 'count': 2683, 'min': 0.0051492155, 'max': 0.99870497, 'sumsqr': 1482.5704344471}}, {'ke

In [25]:
date="2021-05-22"

print( dataofday(date))

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_keywords
Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords
[]


In [26]:
def get_data_summary(db="covid",viewType="month",startkey="2020-10",mode = "mean"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return {}
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="month"):
        view = View(ddoc, 'view_month')
    elif(viewType=="zone"):
        view = View(ddoc,"view_zone")
    else:
        view = View(ddoc, 'view_time')
    result = {}
    for row in view(limit=100,reduce=True,group=True,startkey=startkey)['rows']:
        if(mode=="mean"):
            result[row['key']] = round(row['value']['sum']/row['value']['count'],6)
        elif(mode=="count"):
            result[row['key']] = row['value']['count']
    return result

In [30]:
def get_data_summary(db="covid",viewType="month",startkey="2020-10",mode = "mean"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return {}
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="month"):
        view = View(ddoc, 'view_month')
    elif(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="zone month"):
        view=View(ddoc, 'view_zone_month')
    else:
        view = View(ddoc, 'view_time')
    result = {}
    for row in view(limit=100,reduce=True,group=True,startkey=startkey)['rows']:
        if(mode=="mean"):
            result[row['key']] = round(row['value']['sum']/row['value']['count'],6)
        elif(mode=="count"):
            result[row['key']] = row['value']['count']
    return result

def get_data(n=100,db="covid",viewType="day",startkey="2021-05-01"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return []
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="day"):
        view = View(ddoc, 'view_time')
    else:
        return "Does not support viewType"+viewType
    result = []
    for row in view(limit=n,reduce=False,group=False,startkey=startkey,include_docs=True)['rows']:
        result.append(row)
    return result


In [29]:
print(get_data_summary(viewType='view_time'))

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid
{'2020-10-02': 0.693379, '2020-10-04': 0.501213, '2020-10-07': 0.679913, '2020-10-12': 0.191182, '2020-10-16': 0.548117, '2020-10-17': 0.889036, '2020-10-20': 0.34919, '2020-10-21': 0.964741, '2020-10-22': 0.713141, '2020-10-23': 0.914453, '2020-10-24': 0.479186, '2020-10-27': 0.66142, '2020-10-28': 0.895595, '2020-11-01': 0.5257, '2020-11-02': 0.239922, '2020-11-03': 0.949064, '2020-11-05': 0.211351, '2020-11-08': 0.873696, '2020-11-10': 0.477794, '2020-11-11': 0.8717, '2020-11-13': 0.931579, '2020-11-14': 0.959367, '2020-11-16': 0.52775, '2020-11-18': 0.703877, '2020-11-19': 0.264016, '2020-11-20': 0.909923, '2020-11-22': 0.636922, '2020-11-23': 0.591639, '2020-11-24': 0.926269, '2020-11-25': 0.407742, '2020-11-27': 0.916552, '2020-12-01': 0.755786, '2020-12-02': 0.878444, '2020-12-03': 0.65022, '2020-12-05': 0.883682, '2020-12-08': 0.743107,